In [1]:
from pprint import pprint
import json
import time
import sys
import argparse
import praw
import pprint
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk import LancasterStemmer
from gensim import corpora, models, similarities
from collections import defaultdict
from pprint import pprint
#This is recommended when using gensim 
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import re
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.naive_bayes import MultinomialNB
from random import sample

/Users/taftmphee/anaconda/lib/python2.7/site-packages/numpy/lib/utils.py:95: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)
/Users/taftmphee/anaconda/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)


In [2]:
r = praw.Reddit(user_agent='marytest 0.2')

Version 3.1.0 of praw is outdated. Version 3.2.0 was released 4 days ago.


In [52]:
submissions = r.get_subreddit('Nerd_Corner').get_hot(limit=5)
for submission in submissions:
    print "Title: ", submission.title
    print "Text: ", submission.selftext
    print "Score: ", submission.score
    print "---------------------------------\n"

Title:  we talk about some news would like to here what yall say about it
Text:  
Score:  1
---------------------------------

Title:  Guys, I'm out here tryna get some of these nerd points for kiss cartoon. It's pretty cool actually.
Text:  
Score:  2
---------------------------------

Title:  I've officially become an official nerd
Text:  I'm now addicted to reddit and laugh uncontrollably at jokes like this: 

[Kilobyte](http://imgs.xkcd.com/comics/kilobyte.png)
Score:  1
---------------------------------

Title:  Why nerd culture must die
Text:  
Score:  1
---------------------------------

Title:  Funny Nerd Chase (Unicorn City)
Text:  
Score:  1
---------------------------------



In [53]:
subreddit = r.get_subreddit("astrophysics")

In [54]:
print subreddit.display_name
print subreddit.title

astrophysics
Welcome to nerd heaven


In [55]:
print subreddit.description

**FREQUENTLY ASKED QUESTIONS**


Q. Should I pursue astrophysics?

A. No.


Q. What school should I choose?

A. School is a waste of money.


Q. What jobs are there?

A. None. They're all taken. Do something else.


Q. What's a good book to read?

A. What do I look like, a librarian? Next question please.


Q. Is this a place for discussing my college plans and career goals?

A. NO!!


In [12]:
nerdobject = r.search('nerd', subreddit=None, sort=None, limit=60)
#started with 50

In [13]:
allsubmissions = {}
#dict to hold the titles and flat comments

In [14]:
def get_submissions(allsubmissions, nerdobject):
#limit 10 means max of 10 requests to more comments, and threshold means each call must get at least 10 more
#takes about 2 seconds each so can last a long time depending on parameters, max number loaded initally is 200
    for submission in nerdobject:
        print submission
        #submission.all_comments_flat
        submission.replace_more_comments(limit=5, threshold=10)
        flatsub = praw.helpers.flatten_tree(submission.comments)
        allsubmissions[submission.id] = flatsub


In [15]:
#takes a while
get_submissions(allsubmissions, nerdobject)

904 :: We make music with computers, Pegboard Nerds AMA
1240 :: IAmA Naked News anchor who writes a segment called the Naked Nerd.  I...
3198 :: (27/M/170) Transormation WoW nerd to exercise nerd
133 :: What do you dislike about nerd/geek culture?
762 :: Sean "Day[9]" Plott Talks Starcraft, eSports and Nerd Pride - Forbes
1344 :: Is the social category of "nerd" a necessarily modern phenomenon? Wha...
840 :: AMA Request: The Angry Video Game Nerd
37 :: A common argument was that Anti-GG SJWs were bullying overweight Gamers...
734 :: 2 Girls 1 Nerd
144 :: What was your most memorable 'nerd moment'?
361 :: [SJW] Head of CBI - "Games need 'artistic' designers, too many spotty ...
51 :: Why do women wear " I love Nerds" T-shirts when they would clearly be t...
116 :: I am Curtis Armstrong - Actor and Co-Executive Producer of King of the...
446 :: Nerd Zen: You're shit at everything until you work hard enough to get ...
1288 :: Today I saved a brother nerd from a life of shame and possibly 

In [35]:
#get the submission ids
submissionids = allsubmissions.keys()
print len(submissionids)

60


In [32]:
#now get the comment ids
commentids = []
def get_commentids(commentids, submissionids, allsubmissions):
    x = 0
    for item in submissionids:
        index = item
        for comment in allsubmissions[index]:
            unique = str(comment.id)
            commentids.append(unique)


In [36]:
#now get all the commentids
get_commentids(commentids, submissionids, allsubmissions)

In [37]:
print len(commentids)

9407


In [104]:
#make a dataframe to hold the comment body, score and author
data = pd.DataFrame(index = commentids, columns=['text', 'author', 'score', 'submission_id', 'nerdy'])

In [38]:
#now do a dataframe that is less granular, aggregated by submission_id
aggregatedata = pd.DataFrame(index = submissionids, columns=['text', 'nerdy'])

In [39]:
commentlist = {}
for item in submissionids:
    commentlist =[]
    for comment in allsubmissions[item]:
        commentlist.append(comment)    
    aggregatedata.ix[item]['text'] = comment.body.encode('utf-8')
    aggregatedata.ix[item]['nerdy'] = 1

In [40]:
aggregatedata.head()

,text,nerdy
17235m,"Yep, we all have those days when our hair isn'...",1
31g2lf,Because some Americans have inferiority comple...,1
jcute,is their self esteem low maybe?,1
11sg43,That's fucking badass,1
2tkfr0,"Oh you're completely right, but bullies don't ...",1


In [41]:
#save this as well
aggregatedata.to_csv(path_or_buf='/Users/taftmphee/Desktop/aggregatefile.csv')

In [42]:
aggregatedata = pd.DataFrame.from_csv(path='/Users/taftmphee/Desktop/aggregatefile.csv')

In [69]:
def get_comments(data, submissionids, allsubmissions, nerdy):
    for item in submissionids:
        for comment in allsubmissions[item]:
        #print comment.body
            try:
                name = comment.author.name
            except:
                name = "unknown"
            unique = str(comment.id)
            data.ix[unique]['text'] = comment.body.encode('utf-8')
            data.ix[unique]['author'] = name.encode('utf-8')
            data.loc[unique]['score'] = comment.score
            data.loc[unique]['submission_id'] = item.encode('utf-8')
            data.loc[unique]['nerdy'] = nerdy

In [150]:
get_comments(data, submissionids, allsubmissions, 1)

In [151]:
data.head(10)

,text,author,score,submission_id,nerdy
c81hh9p,"When I finish work everyday, I am excited to g...",SeriousBlack,113,17235m,1
c81j055,I do. Nothing feels sexier to me than eight l...,katcurtis,230,17235m,1
c81kdhw,Eight layers of clothing. Looks like you found...,GonzaCantSleep,51,17235m,1
c81gvlu,I'm going to assume most of your viewers are m...,VanillaBraun,370,17235m,1
c81h21m,Thanks for starting this off! Good question- ...,katcurtis,617,17235m,1
c81hak6,I'm assuming that because Naked News is someth...,Boogaloooooooooooooo,192,17235m,1
c81hcvl,From watching the video on the homepage of the...,the_bitch,445,17235m,1
c81hysn,I'll agree with you on the way our site looks....,katcurtis,228,17235m,1
c81ia42,The preview video isnt working on my tablet so...,BabySinister,75,17235m,1
c81ikrh,Isn't that the entire point of the show?,CitizenJake,21,17235m,1


In [152]:
#save the file
data.to_csv(path_or_buf='/Users/taftmphee/Desktop/nerdyfile.csv')

In [43]:
data = pd.DataFrame.from_csv(path='/Users/taftmphee/Desktop/nerdyfile.csv')

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9283 entries, c81hh9p to cs2t45n
Data columns (total 5 columns):
text             9281 non-null object
author           9280 non-null object
score            9280 non-null object
submission_id    9280 non-null object
nerdy            9279 non-null float64
dtypes: float64(1), object(4)
memory usage: 435.1+ KB


In [59]:
subredditlist=[]
#now get a bunch of non-nerdy subreddits
subreddits = r.get_popular_subreddits(limit = 10)
#[str(x) for x in subreddits]
for subreddit in subreddits:
    print subreddit
    subredditlist.append(subreddit.display_name)

funny
pics
AdviceAnimals
todayilearned
gifs
aww
WTF
videos
gaming
AskReddit


In [60]:
print subredditlist

[u'funny', u'pics', u'AdviceAnimals', u'todayilearned', u'gifs', u'aww', u'WTF', u'videos', u'gaming', u'AskReddit']


In [61]:
#dict to hold the new submissions
allcoolsubmissions = {}

In [62]:
#from them take the non-nerdy submissions
for subreddit in subredditlist:
    print subreddit
    next = r.get_subreddit(subreddit).get_hot(limit=5)
    #now that we have the hot submissions for that subreddit call the function to get the full submission and comments
    get_submissions(allcoolsubmissions, next)


funny
461 :: Subreddit Of The Month [August]: /r/askashittydoctor, "When you can't ...
5512 :: Parenting Idea
3213 :: The other languages on this hot coffee sleeve are fake-French and fak...
5141 :: WRONG LEVVEEERRRR!!!!!
2327 :: We are the public education system
pics
6133 :: At this point you might as well be nude [NSFW]
5438 :: We put Danny Trejo in a dog costume for our new ad. He loved it.
7352 :: Drought in California...people used to swim here
2200 :: Amazing Cosplay
3093 :: 2nd in my paintings of fallen angels, Kokabiel, Angel of the Stars
AdviceAnimals
168 :: /r/AdviceAnimals rules: Please read before submitting or commenting
6332 :: So tired of the in my day argument
4521 :: Don't talk it up if you don't mean it.
5530 :: Game mode set on Easy and I still can't win.
1530 :: I'll Keanu Leaves now.
todayilearned
2109 :: Please copy/paste the relevant text from your source in the comments ...
7175 :: TIL that a father was denied access to see his premature twins in the...
5370 ::

In [63]:
#get the commentids
print len(allcoolsubmissions)
coolsubmissionids = allcoolsubmissions.keys()
print coolsubmissionids

In [65]:
coolcommentids =[]

In [66]:
#now get all the commentids
get_commentids(coolcommentids, coolsubmissionids, allcoolsubmissions)

In [67]:
#make a dataframe to hold the comment body, score and author
cooldata = pd.DataFrame(index = coolcommentids, columns=['text', 'author', 'score', 'submission_id', 'nerdy'])

In [70]:
get_comments(cooldata, coolsubmissionids, allcoolsubmissions, 0)

In [71]:
cooldata.head()

,text,author,score,submission_id,nerdy
cu7saqv,Reminds me of a story I read on Reddit about a...,DaShazam,109,3hibut,0
cu7wk8s,I would have opened a competing bake sale that...,TheGillos,69,3hibut,0
cu7xbpv,Ahhhh the American dream.,Outmodeduser,28,3hibut,0
cu7y39f,Sounds like something frank reynolds would do.,ralph122030,5,3hibut,0
cu7yazk,Jack Donaghy you magnificent bastard!,StepUpYourLife,3,3hibut,0


In [72]:
cooldata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9991 entries, cu7saqv to cu7tma1
Data columns (total 5 columns):
text             9991 non-null object
author           9991 non-null object
score            9991 non-null object
submission_id    9991 non-null object
nerdy            9991 non-null object
dtypes: object(5)
memory usage: 468.3+ KB


In [73]:
#save this data
cooldata.to_csv(path_or_buf='/Users/taftmphee/Desktop/coolfile.csv')

In [74]:
cooldata = pd.DataFrame.from_csv(path='/Users/taftmphee/Desktop/coolfile.csv')

In [75]:
#merge the two files for a granular dataframe
fulldata = pd.merge(data, cooldata, how='outer')

In [76]:
fulldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19275 entries, 0 to 19274
Data columns (total 5 columns):
text             19273 non-null object
author           19271 non-null object
score            19271 non-null object
submission_id    19271 non-null object
nerdy            19269 non-null float64
dtypes: float64(1), object(4)
memory usage: 903.5+ KB


In [77]:
#not sure how I lost the commentids here but can always look them back up
fulldata.head()

,text,author,score,submission_id,nerdy
0,"When I finish work everyday, I am excited to g...",SeriousBlack,113,17235m,1
1,I do. Nothing feels sexier to me than eight l...,katcurtis,230,17235m,1
2,Eight layers of clothing. Looks like you found...,GonzaCantSleep,51,17235m,1
3,I'm going to assume most of your viewers are m...,VanillaBraun,370,17235m,1
4,Thanks for starting this off! Good question- ...,katcurtis,617,17235m,1


In [78]:
fulldata.to_csv(path_or_buf='/Users/taftmphee/Desktop/fulldata.csv')

In [79]:
fulldata.ix[13]['text']

'[deleted]'

In [112]:
fulldata = pd.DataFrame.from_csv(path='/Users/taftmphee/Desktop/fulldata.csv')
#get rid of deleted comments
fullestdata = fulldata[fulldata.text != '[deleted]']

In [113]:
fullestdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18946 entries, 0 to 19274
Data columns (total 5 columns):
text             18944 non-null object
author           18942 non-null object
score            18942 non-null object
submission_id    18942 non-null object
nerdy            18940 non-null float64
dtypes: float64(1), object(4)
memory usage: 888.1+ KB


In [84]:
#now make a dataframe for cool data at the less granular level, aggregated by submission_id
coolaggregatedata = pd.DataFrame(index = coolsubmissionids, columns=['text', 'nerdy'])

In [85]:
for item in coolsubmissionids:
    coolcommentlist =[]
    for comment in allcoolsubmissions[item]:
        coolcommentlist.append(comment)    
    coolaggregatedata.ix[item]['text'] = comment.body.encode('utf-8')
    coolaggregatedata.ix[item]['nerdy'] = 0

In [86]:
coolaggregatedata.head()

,text,nerdy
3hibut,Did anyone here actually look up what intersec...,0
3hi3om,"> to cease from maintaining, practicing, or using",0
3hi6uw,[Comment about size of his balls]\n\n\n,0
3hhn9m,That's stupid. He only moves his left hand bec...,0
3hhj68,Fuck me mate!,0


In [88]:
aggregatesubmissionids = []

In [89]:

for item in coolsubmissionids:
    aggregatesubmissionids.append(item)
for item in submissionids:
    aggregatesubmissionids.append(item)

In [91]:
#learned a new way to combine data frames
frames = (aggregatedata, coolaggregatedata)
aggregatefulldata = pd.concat(frames)

In [90]:
len(aggregatesubmissionids)

110

In [92]:
aggregatefulldata.info()
#this is 60 nerdy submissions and 50 cool ones but similar total document length because the comments are balanced

<class 'pandas.core.frame.DataFrame'>
Index: 110 entries, 17235m to 3hicc7
Data columns (total 2 columns):
text     110 non-null object
nerdy    110 non-null object
dtypes: object(2)
memory usage: 2.6+ KB


In [93]:
aggregatefulldata.head()

,text,nerdy
17235m,"Yep, we all have those days when our hair isn'...",1
31g2lf,Because some Americans have inferiority comple...,1
jcute,is their self esteem low maybe?,1
11sg43,That's fucking badass,1
2tkfr0,"Oh you're completely right, but bullies don't ...",1


In [114]:
#fullestdata.isnull().sum()
fullestdata.dropna(inplace = True)

/Users/taftmphee/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [115]:
fullestdata.head()

,text,author,score,submission_id,nerdy
0,"When I finish work everyday, I am excited to g...",SeriousBlack,113,17235m,1
1,I do. Nothing feels sexier to me than eight l...,katcurtis,230,17235m,1
2,Eight layers of clothing. Looks like you found...,GonzaCantSleep,51,17235m,1
3,I'm going to assume most of your viewers are m...,VanillaBraun,370,17235m,1
4,Thanks for starting this off! Good question- ...,katcurtis,617,17235m,1


In [116]:
#start running models on the more granular data 
X = fullestdata['text']
y = fullestdata['nerdy'].astype(int)


In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [118]:
#start with a count vectorizer
vectorizer = CountVectorizer()

#do a fit transform on the training set
CVX_train = vectorizer.fit_transform(X_train)
print len(vectorizer.get_feature_names())
print vectorizer.get_feature_names()[3000:3010]

#just transform the testing set
CVX_test = vectorizer.transform(X_test)
print CVX_test.shape

25020
[u'biddle', u'biden', u'biding', u'bieber', u'bien', u'bienvenida', u'big', u'bigass', u'bigbootay', u'bigger']
(4735, 25020)


In [119]:
ll=vectorizer.get_feature_names()
for i in range(75,100):
    print "{:25s} {:50s}".format(ll[i], ll[i+500])
len(ll)
##going to need to remove the comment ids from this if we don't want association with nerdy commenters?

103                       3am                                               
1044                      3b3d5668                                          
105                       3bde631fbc88e3aa                                  
1051                      3d                                                
106                       3dmark                                            
10608739                  3dmzbf                                            
1067                      3ds                                               
108424                    3enpgf                                            
10944                     3f                                                
109awg                    3fjqon5_euk                                       
10c                       3g                                                
10gb                      3h61d1                                            
10hu4i                    3hak3an2q2a                                       

25020

In [120]:
#now a logistic regression model 
CVlogreg = LogisticRegression(C=1000)
CVlogreg.fit(CVX_train, y_train)
CVyhat = CVlogreg.predict(CVX_test)

In [121]:
CVlogreg.score(CVX_test, y_test)

0.70179514255543818

In [122]:
pd.crosstab(CVyhat, y_test, rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,1812,734
1,678,1511


In [123]:
#surprisingly better
multi = MultinomialNB().fit(CVX_train, y_train)
print multi.score(CVX_test, y_test)
pd.crosstab(y_test, multi.predict(CVX_test), rownames=["Actual"], colnames=["Predicted"])

0.751847940866


Predicted,0,1
Actual,,
0,1846,644
1,531,1714


In [124]:
#now tfidf--better score for ngrams of 2 than 3--try this again after cleaning text
ngr = (1, 2)
tfidf = TfidfVectorizer(ngram_range = ngr, stop_words = "english")

In [125]:
TFX_train = tfidf.fit_transform(X_train)
TFX_test = tfidf.transform(X_test)
print TFX_train.shape

(14205, 179763)


In [126]:
train_words = tfidf.get_feature_names()
#print train_words

In [127]:
#tfidf returns real numbers (not counts) so start with a logistic regression model 
TFlogreg = LogisticRegression(C=1000)
TFlogreg.fit(TFX_train, y_train)
TFyhat = TFlogreg.predict(TFX_test)

In [128]:
TFlogreg.score(TFX_test, y_test)

0.75586061246040126

In [129]:
pd.crosstab(TFyhat, y_test, rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,1918,584
1,572,1661


In [130]:
#not a lot of extra juice from tfidf
multi = MultinomialNB().fit(TFX_train, y_train)
print multi.score(TFX_test, y_test)
pd.crosstab(y_test, multi.predict(TFX_test), rownames=["Actual"], colnames=["Predicted"])

0.768532206969


Predicted,0,1
Actual,,
0,1862,628
1,468,1777


In [274]:
#try tfidf at the submission level
aggX = aggregatefulldata['text']
aggy = aggregatefulldata['nerdy'].astype(int)


In [275]:
aggX_train, aggX_test, aggy_train, aggy_test = train_test_split(aggX, aggy)

In [289]:
#still better score for ngrams of 2 than 3
ngr = (1, 2)
aggtfidf = TfidfVectorizer(ngram_range = ngr, stop_words = "english")

In [290]:
aggTFlogreg = LogisticRegression(C=1000)
aggTFX_train = aggtfidf.fit_transform(aggX_train)
aggTFX_test = aggtfidf.transform(aggX_test)

In [291]:
aggTFlogreg.fit(aggTFX_train, aggy_train)
aggTFyhat = aggTFlogreg.predict(aggTFX_test)

In [292]:
aggTFlogreg.score(aggTFX_test, aggy_test)

0.6071428571428571

In [131]:
#get rid of links 
#url_regex = re.compile(r"""(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>\[\]]+|\(([^\s()<>\[\]]+|(\([^\s()<>\[\]]+\)))*\))+(?:\(([^\s()<>\[\]]+|(\([^\s()<>\[\]]+\)))*\)|[^\s`!(){};:'".,<>?\[\]]))""")
#url_regex2 = re.compile('http[s]? : //(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
#url_regex3 = re.compile(r'(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))')
url_regex4 = re.compile(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''')

In [132]:
cleanX = []
bads = []

In [133]:
def clean(cleanX, X):
    #comments that are links
    for item in X:
        m = url_regex4.match(item)
        #clean out some punctuation to see if there are more links
        cleaner_string = [re.sub("[)/('?]", "", item)]
        if m:
            print m.group()
            bads.append(m.group())
            cleanX.append('URL_OMITTED')
        #elif item.startswith('http : //'): 
        #   cleanX.append('URL_OMITTED')
        #elif item.startswith('www'): 
        #   cleanX.append('URL_OMITTED')
        elif str(cleaner_string).startswith('http :'):
            print cleaner_string
            cleanX.append('URL_OMITTED')
            
        elif str(cleaner_string).startswith('www'): 
            print cleaner_string
            cleanX.append('URL_OMITTED')
        else:
            cleanX.append(item)
    print len(bads)

In [134]:
clean(cleanX, X)

http://i.imgur.com/YlE2U.gif
http://www.youtube.com/watch?v=5BrkmIqUXck
http://vimeo.com/15949396
www.nakednews.com
http://youtu.be/tN3qn92R0SE
https://www.youtube.com/watch?v=C4wBLUBa8YI
http://i.imgur.com/do2iD.gif
http://news.ku.edu/2014/01/15/study-shows-high-school-athletes-performed-better-school-persisted-graduation-more-non
http://www.reddit.com/r/malefashionadvice/comments/1kcfjs/mfa_bag_guide_20/
http://i.imgur.com/IOhCQcx.jpg
http://youtu.be/icpSKRdQsJE
http://s3.amazonaws.com/kym-assets/entries/icons/original/000/006/151/tumblr_lltzgnHi5F1qzib3wo1_400.jpg?1308698386
http://www.reddit.com/r/gonewild/comments/aqjl6/you_guys_are_fucking_creepy/
http://www.youtube.com/watch?v=7QPMvj_xejg
www.mars2112.com
http://i.imgur.com/qYVEFYm.jpg
http://i.imgur.com/XzpWeWw.png
https://i.imgur.com/2o6lNXc.jpg
http://i.imgur.com/XzpWeWw.png
http://i.imgur.com/ysWjUNq.png
http://i.imgur.com/ysWjUNq.png
http://meaningness.com/metablog/geeks-mops-sociopaths
http://patri-archie-comics.tumblr.com

In [135]:
#print cleanX
cleanX.count('URL_OMITTED')

158

In [136]:
cleanerX = []

In [137]:
#links inside the comments
def cleaner(cleanerX, cleanX):
    for item in cleanX:
    #tokenize it to find links in comments:
        item = unicode(item, 'utf-8')
        #clean out some punctuation to see if there are more links
        cleaner_string = str([re.sub("[)/(/'?]", " ", item)])
        if cleaner_string.startswith('http'):
            print cleaner_string
            cleanerX.append('URL_OMITTED')
        else:
            tokens = nltk.word_tokenize(cleaner_string.lower())
            string = []
            for token in tokens:
                m = url_regex4.match(token)
                if m:
                    print m.group()
                    bads.append(m.group())
                    token = 'URL_OMITTED'
                    string.append(token)
                elif token.startswith('www'):
                    print token
                    string.append('URL_OMITTED')
                else:
                    string.append(token)
            #rejoin and return
            cleanerX.append(" ".join(string))

In [138]:
cleaner(cleanerX, cleanX)

www.nakednews.com
www.youtube.com
www.katherinecurtis.com
www.katherinecurtis.com
www.katherinecurtis.com
www.katherinecurtis.com
www.youtube.com
www.reddit.com
www.reddit.com
www.reddit.com
www.reddit.com
www.reddit.com
www.reddit.com
www.youtube.com
www.vegalicious.org
www.nakednews.com
www.twitpic.com
www.youtube.com
www.youtube.com
www.thegreenhead.com
www.reddit.com
www.reddit.com
www.youtube.com
www.youtube.com
www.knbc.com
www.knbc.com
www.reddit.com
www.t-nation.com
www.t-nation.com
www.dailymail.co.uk
www.navy.mil
www.explainxkcd.com
www.xkcd.com
www.reddit.com
www.reddit.com
www.reddit.com
www.youtube.com
www.youtube.com
www.youtube.com
www.youtube.com
www.flickr.com
www.ironicsans.com
www.geocaching.com
www.geocaching.com
www.lsc.org
www.nysci.org
www.whitecastle.com
www.moma.org
www.moma.org
www.amnh.org
www.toyspaceny.com
www.thecompleatstrategist.com
www.yelp.com
www.sakebardecibel.com
www.moccany.com
www.thinkgeek.com
www.ninjanewyork.com
www.youtube.com
www.reddit.com
w

In [139]:
len(bads)

795

In [157]:
#now try it cleaned up and stemmed
def clean_and_stem(text):
    '''This is a simple function to clean up text. This function will convert text to lower case, tokenize the test
    into words, eliminate stop words, and then stem the text'''
    
    #use the sklearn list of stop words
    stop_words = ENGLISH_STOP_WORDS
    
    #first get rid of special characters except apostrophes
    clean_text = re.sub("[^A-Za-z']", " ", text)
    
    #try the Lancaster stemmer - but there are others in NLTK
    stemmer = nltk.stem.LancasterStemmer()
    
    #now tokenize
    tokens = nltk.word_tokenize(str(clean_text))
                                #lower())
    
    #remove stop words
    clean_tokens = [word for word in tokens if word not in stop_words]
    
    #stem
    stem_tokens = [stemmer.stem(token) for token in clean_tokens]
    
    #rejoin and return
    return " ".join(stem_tokens)
    #(cleaner_tokens)
    #(stem_tokens)

In [158]:
cleanestX = []
for item in cleanerX:
    #item = unicode(item, 'utf-8')
    thing = clean_and_stem(item)
    cleanestX.append(thing)

In [159]:
print len(cleanestX)
print len(y)

18940
18940


In [162]:
cleanX_train, cleanX_test, cleany_train, cleany_test = train_test_split(cleanestX, y)
#print cleanX_test

In [163]:
#ngrams of 2 still slightly better
ngr = (1, 2)
cleantfidf = TfidfVectorizer(ngram_range = ngr)
                             #stop_words = "english")

In [164]:
cleanX_train = cleantfidf.fit_transform(cleanX_train)
cleanX_test = cleantfidf.transform(cleanX_test)
print cleanX_train.shape
print cleanX_test.shape

(14205, 162364)
(4735, 162364)


In [165]:
cleanlogreg = LogisticRegression(C=1000)
cleanlogreg.fit(cleanX_train, cleany_train)
cleanyhat = cleanlogreg.predict(cleanX_test)

In [166]:
#score decreases with links removed
cleanlogreg.score(cleanX_test, cleany_test)

0.75881731784582895

In [167]:
pd.crosstab(cleanyhat, cleany_test, rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,1932,554
1,588,1661


In [168]:
#also worse with links removed
multi = MultinomialNB().fit(cleanX_train, cleany_train)
print multi.score(cleanX_test, cleany_test)
pd.crosstab(cleany_test, multi.predict(cleanX_test), rownames=["Actual"], colnames=["Predicted"])

0.756494192186


Predicted,0,1
Actual,,
0,1821,699
1,454,1761


In [293]:
cleanaggX = []

In [294]:
#again at the submission level
clean(cleanaggX, aggX)

https://www.youtube.com/watch?v=FWhZ2XS08aQ
https://youtu.be/iNSt3wJXZk0?t=113
797


In [300]:
cleaneraggX = []

In [301]:
cleaner(cleaneraggX, cleanaggX)

www.reddit.com
www.reddit.com
www.notehub.org
www.reddit.com
www.reddit.com
www.reddit.com:443
www.notehub.org
www.reddit.com
www.reddit.com:443
www.notehub.org
www.reddit.com
www.reddit.com
www.reddit.com
www.reddit.com


In [297]:
cleanestaggX = []
for item in cleaneraggX:
    #item = unicode(item, 'utf-8')
    thing = clean_and_stem(item)
    cleanestaggX.append(thing)

In [311]:
cleanaggX_train, cleanaggX_test, cleanaggy_train, cleanaggy_test = train_test_split(cleanestaggX, aggy)

In [312]:
#ngrams of 2 still slightly better
ngr = (1, 2)
cleanaggtfidf = TfidfVectorizer(ngram_range = ngr)
                             #stop_words = "english")

In [313]:
cleanaggX_train = cleanaggtfidf.fit_transform(cleanaggX_train)
cleanaggX_test = cleanaggtfidf.transform(cleanaggX_test)

In [314]:
cleanagglogreg = LogisticRegression(C=1000)
cleanagglogreg.fit(cleanaggX_train, cleanaggy_train)
cleanaggyhat = cleanagglogreg.predict(cleanaggX_test)

In [315]:
print cleanagglogreg.score(cleanaggX_test, cleanaggy_test)
pd.crosstab(cleanaggyhat, cleanaggy_test, rownames=["Actual"], colnames=["Predicted"])

0.535714285714


Predicted,0,1
Actual,,
0,6,10
1,3,9


In [169]:
#let's lemm instead of stem
def clean_and_lemm(text):
    '''This is a simple function to clean up text. This function will convert text to lower case, tokenize the test
    into words, eliminate stop words, and then stem the text'''
    
    #use the sklearn list of stop words
    stop_words = ENGLISH_STOP_WORDS
    
    
    #''WordNet superficially resembles a thesaurus, in that it groups words together based on their meanings.
    #BUT words that are found in close proximity to one another in the network are semantically disambiguated
    #AND WordNet labels the semantic relations among words, whereas the groupings of words in a 
    #thesaurus does not follow any explicit pattern other than meaning similarity''
    lemmer = nltk.stem.WordNetLemmatizer()
    
    #get rid of everything except letters spaces and apostrophes
    clean_text = re.sub("[^A-Za-z']", " ", text)
    
    #tokenize
    tokens = nltk.word_tokenize(clean_text)
    
    #remove stop words
    clean_tokens = [word for word in tokens if word not in stop_words]
    
    #stem
    lemm_tokens = [lemmer.lemmatize(token) for token in clean_tokens]
    
    #rejoin and return
    return " ".join(lemm_tokens)
    

In [170]:
lemmedX = []
for item in cleanerX:
    #item = unicode(item, 'utf-8')
    thing = clean_and_lemm(item)
    lemmedX.append(thing)

In [179]:
lemmX_train, lemmX_test, lemmy_train, lemmy_test = train_test_split(lemmedX, y)

In [180]:
#better with ngrams of 2 still
ngr = (1, 2)
lemmtfidf = TfidfVectorizer(ngram_range = ngr, stop_words = "english")

In [181]:
lemmX_train = lemmtfidf.fit_transform(lemmX_train)
lemmX_test = lemmtfidf.transform(lemmX_test)

In [182]:
lemmlogreg = LogisticRegression(C=1000)
lemmlogreg.fit(lemmX_train, lemmy_train)
lemmyhat = lemmlogreg.predict(lemmX_test)

In [183]:
#lemmatizing is better than stemming
lemmlogreg.score(lemmX_test, lemmy_test)

0.75227032734952481

In [184]:
pd.crosstab(lemmyhat, lemmy_test, rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,1964,626
1,547,1598


In [185]:
#not predicting as well as before
multi = MultinomialNB().fit(lemmX_train, lemmy_train)
print multi.score(lemmX_test, lemmy_test)
pd.crosstab(lemmy_test, multi.predict(lemmX_test), rownames=["Actual"], colnames=["Predicted"])

0.765364308342


Predicted,0,1
Actual,,
0,1877,634
1,477,1747


In [ ]:
##below this are unfinished models and the call for selected data

In [483]:
#better with ngrams of 2 still
ngr = (1, 2)
cosinetfidf = TfidfVectorizer(ngram_range = ngr, stop_words = "english")

In [481]:
cosineX_test = cosinetfidf.fit_transform(cleanestselectX)
cosineaggX_test = cosinetfidf.fit_transform(cleanaggX)    

In [480]:
print cleanaggX_train.shape
print shapedcosineX_test.shape


(82, 1477)
(82, 20082)


In [482]:
shapedcosineX_test = cosineX_test[:82]

In [484]:
#print cosine_similarity(cleanaggX_train, shapedcosineX_test)
#gives ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 1477 while Y.shape[1] == 20082
##this takes either row of matrix or matrix itself
##tfidf transform on the new text to use this to classify

In [209]:
#now try with POS tagging
POScleanerX =[]
n = 0
for i, item in enumerate(cleanerX):
    tag_list = []
    clean_item = re.sub("[^A-Za-z']", " ", item)
    if i % 1000 == 0:
        print '.',
    nltk_tokens = nltk.word_tokenize(str(clean_item))
    
    #use the sklearn list of stop words
    stop_words = ENGLISH_STOP_WORDS
    
    #remove stop words
    clean_tokens = [word for word in nltk_tokens if word not in stop_words]
    
    POScleanerX.append(nltk.pos_tag(clean_tokens))
print len(POScleanerX)

. . . . . . . . . . . . . . . . . . . 18940


In [217]:
NEcleanerX =[]
n = 0
for i, item in enumerate(POScleanerX):
    if i % 1000 == 0:
        print '.',
    ner_list = nltk.ne_chunk(item)
NEcleanerX.append(ner_list)

. . . . . . . . . . . . . . . . . . .


In [219]:
print NEcleanerX
print ner_list
#cross validate the model you like

[Tree('S', [("u'elbows", 'NNS'), ('passed', 'VBD'), ('edge', 'NN'), ('doesn', 'NN'), ('t', 'NN'), ('count', 'NN'), ("'", "''")])]
(S u'elbows/NNS passed/VBD edge/NN doesn/NN t/NN count/NN '/'')


In [320]:
#not enough time to get into this unfortunately
help(nltk.ne_chunk)
for p in ner_list:
    nltk.help.upenn_tagset(p[1])

Help on function ne_chunk in module nltk.chunk:

ne_chunk(tagged_tokens, binary=False)
    Use NLTK's currently recommended named entity chunker to
    chunk the given list of tagged tokens.

NNS: noun, common, plural
    undergraduates scotches bric-a-brac products bodyguards facets coasts
    divestitures storehouses designs clubs fragrances averages
    subjectivists apprehensions muses factory-jobs ...
VBD: verb, past tense
    dipped pleaded swiped regummed soaked tidied convened halted registered
    cushioned exacted snubbed strode aimed adopted belied figgered
    speculated wore appreciated contemplated ...
NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhuman

In [323]:
#check out this model on selected nerdy data--pulling this data above but leaving the cleanup steps down here
#get some random non-nerdy data to mix up the dataframe
rindex =  np.array(sample(xrange(len(cooldata)), 341))

# get 10 random rows from df
dfr = cooldata.ix[rindex]

In [324]:
dfr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 341 entries, cu7u3xr to cu7y8jz
Data columns (total 5 columns):
text             341 non-null object
author           341 non-null object
score            341 non-null object
submission_id    341 non-null object
nerdy            341 non-null float64
dtypes: float64(1), object(4)
memory usage: 16.0+ KB


In [345]:
#merge the two files
selectfulldata = pd.merge(selectdata, dfr, how='outer')

In [346]:
selectfulldata.head()

,text,author,score,submission_id,nerdy
0,Looks great. Thanks for contributing!,jaur,2,3gmj5h,1
1,thank you!,rasapastasalad,1,3gmj5h,1
2,"Not afaik. Reddit, celeb tweets or other viral...",nobody_from_nowhere,1,3b2qyz,1
3,Original Post referenced from /r/nerdchat by /...,OriginalPostSearcher,2,3gjt3k,1
4,Can you give a location?,Jonfirst,1,3dctzs,1


In [347]:
selectfulldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 919 entries, 0 to 918
Data columns (total 5 columns):
text             919 non-null object
author           919 non-null object
score            919 non-null float64
submission_id    919 non-null object
nerdy            919 non-null float64
dtypes: float64(2), object(3)
memory usage: 43.1+ KB


In [348]:
selectX = selectfulldata['text']
selecty = selectfulldata['nerdy'].astype(int)

In [349]:
#start cleaning up the new X--here's a list for them:
cleanselectX = []

In [350]:
bads = []

In [351]:
clean(cleanselectX, selectX)

http://imgur.com/4Dk1Cyb
http://i.imgur.com/Xm5Ot1h.jpg
http://www.sportspickle.com/wp-content/uploads/2015/02/cards2.jpg
3


In [352]:
cleanerselectX = []

In [353]:
cleaner(cleanerselectX, cleanselectX)

www.reddit.com
www.reddit.com
www.abqstylehomes.com
www.abqstylehomes.com
www.abqstylehomes.com
www.amazon.com
www.amazon.com
www.elainecunningham.com
www.reddit.com
www.google.com
www.amazon.com
www.nafsa.org
www.amazon.com
www.sports-reference.com
www.gammathon.com\ni
www.facebook.com
www.reddit.com
www.reddit.com
www.myscizzle.com
www.myscizzle.com
www.oystir.com\nwebinars
www.oystir.com
www.oystir.com
www.obutto.eu
www.gfycat.com
www.youtube.com
www.imdb.com
www.washingtonpost.com
www.google.com
www.bpong.com
www.clarionledger.com
www.youtube.com
www.reddit.com
www.bombjack.org
www.youtube.com


In [354]:
cleanestselectX = []
for item in cleanerselectX:
    #item = unicode(item, 'utf-8')
    thing = clean_and_lemm(item)
    cleanestselectX.append(thing)

In [355]:
selectlemmX_train, selectlemmX_test, selectlemmy_train, selectlemmy_test = train_test_split(cleanestselectX, selecty)

In [356]:
ngr = (1, 2)
selectlemmtfidf = TfidfVectorizer(ngram_range = ngr, stop_words = "english")

In [357]:
selectlemmX_train = selectlemmtfidf.fit_transform(selectlemmX_train)
selectlemmX_test = selectlemmtfidf.transform(selectlemmX_test )

In [358]:
selectlemmlogreg = LogisticRegression(C=1000)
selectlemmlogreg.fit(selectlemmX_train, selectlemmy_train)
selectlemmyhat = selectlemmlogreg.predict(selectlemmX_test)

In [359]:
selectlemmlogreg.score(selectlemmX_test, selectlemmy_test)

0.72173913043478266

In [360]:
pd.crosstab(selectlemmyhat, selectlemmy_test, rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,39,8
1,56,127


In [381]:
#try treating the new X as an independent test set
#can't do this using the tfidf that was trained on the original set unfortunately gives the wrong feature vector
#that model was lemmtfidf
cleanestselectXtest = selectlemmtfidf.transform(cleanestselectX)

In [385]:
cleanestselectyhat = selectlemmlogreg.predict(cleanestselectXtest)
print len(selecty)
#print cleanestselectXtest

919


In [384]:
selectlemmlogreg.score(cleanestselectXtest, selecty)
#this score is biased from the inclusion of test data in the training set--no good

0.92165397170837871

In [222]:
#now trying LDA. each document is a probability distribution over the words
#each comment is a document
stop_list = ENGLISH_STOP_WORDS
documents_without_stops = []
for item in lemmedX:
    t, v = [word for word in item.split(',', 1) if word not in stop_list]
    documents_without_stops.append(t)

In [223]:
frequency = defaultdict(int)
##this is from python library called collections

In [224]:
#print documents_without_stops
for text in documents_without_stops:
    for token in text:
        frequency[token] += 1

In [250]:
texts = []
for text in documents_without_stops:
#tokenizedCorpus = [dictionary2.doc2bow(text) for text in documents_without_stops]
    t = [token for token in text if frequency[token] > 1]
     for line in istream:
        line = line.strip()
        try:
            k, v = line.split(',', 1)
            answer[k.strip()].append(v.strip())
        except ValueError:
    texts.append(t)

In [251]:
#print tokenizedCorpus
print texts

[["u'when", 'finish', 'work', 'everyday', 'excited', 'home', 'work', 'clothes', 'n', 'ndo', 'home', 'clothes', "'"], [u"u'i", u'feel', u'sexier', u'layer', u'clothing', u'love', u'pant', u"'"], [u'layer', u'clothing', u'look', u'like', u'new', u'hit', u'nude', u'novel', u"'"], [u"u'i", u'm', u'going', u'assume', u'viewer', u'men', u'feel', u"'"], [u"u'thanks", u'starting', u'good', u'question', u'right', u'viewer', u'men', u'like', u'program', u'cater', u'demographic', u'fun', u'fall', u'traditionally', u'male', u"''", u'demographic', u'paid', u'talk', u'comic', u'book', u'video', u'game', u'day', u'm', u'wave', u'feminist', u'remember', u'm', u'writing', u'craft', u'segment', u'n', u'n', u'far', u'feel', u'men', u'staring', u'breast', u'like', u'naked', u'wouldn', u't', u'say', u'm', u'exhibitionist', u'don', u't', u'sort', u'thrill', u'people', u'staring', u've', u'relatively', u'comfortable', u'body', u'make', u'happy', u'work', u'field', u'actively', u'trying', u'break', u'stigma',

In [ ]:
#nltk.help.upenn_tagset(‘RB’), or a regular expression, e.g., nltk.help.upenn_brown_tagset(‘NN.*’)

In [259]:
dictionary = corpora.Dictionary(texts)
print "dictionary is ", dictionary
#docs to vectors--'bag of words' approach

dictionary is  Dictionary(13809 unique tokens: [u'fawn', u'unscientific', u'worshiped', u"u'making", u'galactica']...)


In [255]:
#print "Dictionary to token", (dictionary2.token2id)
#this lets you work back more easily from the sparse matrix to see what word 

In [387]:
#LDA = models.ldamodel.LdaModel(texts, id2word=dictionary, num_topics=4, passes = 10, iterations=1000)
#have to have enough passes and iterations to avoide nonconvergence error
#can't get this to work, gives "too many values to unpack" error

In [264]:
for i, topic in enumerate(LDA.print_topics(num_topics = 4, num_words = 10)):
### adjust num_words to see more words associated with that topid
    print "\n\ntopic {:d}:\n".format(i), topic



topic 0:
0.000*ntop + 0.000*management + 0.000*u'hospital + 0.000*pmwiki + 0.000*screwed + 0.000*user + 0.000*skated + 0.000*disclosed + 0.000*thunk + 0.000*min


topic 1:
0.000*bulletin + 0.000*paradox + 0.000*mistake + 0.000*yt + 0.000*u'yo + 0.000*awfulness + 0.000*ftc + 0.000*kawaii + 0.000*youtuber + 0.000*deep


In [317]:
def report_results(clf, my_sparse, my_review):
    #get the probabilities from the model
    res_prob = clf.predict_proba(my_sparse).ravel()
    
    #set up the correct output string
    if clf.predict(my_sparse) == 0:
        result = "fresh"
        
    else:
        result = "rotten"
    
    #print the result making sure the correct probability is used
    print "Your review:\n '", my_review, "'", "\n\nhas a {:5.2f}% chance of being classified in the '{:s}' class".\
    format(res_prob[clf.predict(my_sparse)][0] * 100.0, result)
    print "\n"

In [ ]:
'nerds'
##only like 96 subscribers
'505nerds'
'astrophysics' 
##note the title says it's 'nerd heaven'
'nerdgirls'
#a safe haven for nerd girls"
'labrats'
#lab nerds
'sportsanalytics'
#'for nerds who love sports'
'nerdup'
'LaissezSquares'
#where nerd culture and politics meet
'siliconvalley'
'shut up and build this'
'slashdot'
'karmafleet'

In [330]:
select_subreddits = ('siliconvalley','shut up and build this','505nerds','nerds','astrophysics', 'nerdgirls',\
                     'slashdot','karmafleet','nerdup','LaissezSquares','sportsanalytics','labrats')
#select_subreddits = "'"+"'+'".join(select_subreddits)+"'"

In [331]:
print select_subreddits

('siliconvalley', 'shut up and build this', '505nerds', 'nerds', 'astrophysics', 'nerdgirls', 'slashdot', 'karmafleet', 'nerdup', 'LaissezSquares', 'sportsanalytics', 'labrats')


In [332]:
#dict to hold the new submissions
selectedsubmissions = {}

In [333]:
#from the selected subreddits take top submissions
for subreddit in select_subreddits:
    print subreddit
    next = r.get_subreddit(subreddit).get_hot(limit=10)
    #now that we have the hot submissions for that subreddit call the function to get the full submission and comments
    try:
        get_submissions(selectedsubmissions, next)
    except:
        pass

siliconvalley
111 :: This is the subreddit for the Silicon Valley the place, not the HBO TV...
7 :: Silicon Valley and Diversity, not what you think.
13 :: Google's new parent company Alphabet contains hidden hooli.xyz link in ...
3 :: I'm looking to enter the Tech Industry, but most of my experience is in ...
1 :: Can a business professor move into the tech sector?
3 :: Need advice for starting a career in SV
5 :: Salesforce Makes Strides Toward Gender Equality in Silicon Valley
5 :: 5 Ways to Make Your MBA Work for You -- Even in Silicon Valley
7 :: The Next Wave, Silicon Valley near its end?
31 :: What happens when you talk salary at Google.
shut up and build this
505nerds
3 :: X-Post from r/Albuquerque: Any Gamers with a cool setup?
7 :: MTG, D&D groups?
17 :: Actual Play Podcast based out of Albuquerque
6 :: Looking to unload a bunch of my older comics.
5 :: 505nerds has a new rule.
3 :: PS4: Destiny
3 :: Looking for local 'World of Tanks' players
13 :: Looking for a place to Host

In [334]:
#get the submission ids
print len(selectedsubmissions)
selectedsubmissionids = selectedsubmissions.keys()
#print selectedsubmissionids

107


In [335]:
#list for commentids
selectedcommentids = []

In [336]:
#now get all the commentids
get_commentids(selectedcommentids, selectedsubmissionids, selectedsubmissions)

In [337]:
#make a dataframe to hold the comment body, score and author
selectdata = pd.DataFrame(index = selectedcommentids, columns=['text', 'author', 'score', 'submission_id', 'nerdy'])

In [338]:
get_comments(selectdata, selectedsubmissionids, selectedsubmissions, 1)

In [339]:
selectdata.head()

,text,author,score,submission_id,nerdy
cu3gc5g,Looks great. Thanks for contributing!,jaur,2,3gmj5h,1
cu465f6,thank you!,rasapastasalad,1,3gmj5h,1
csj4wa5,"Not afaik. Reddit, celeb tweets or other viral...",nobody_from_nowhere,1,3b2qyz,1
ctys75c,Original Post referenced from /r/nerdchat by /...,OriginalPostSearcher,2,3gjt3k,1
ct4f7hd,Can you give a location?,Jonfirst,1,3dctzs,1


In [343]:
selectdata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 578 entries, cu3gc5g to cu89hjm
Data columns (total 5 columns):
text             578 non-null object
author           578 non-null object
score            578 non-null int64
submission_id    578 non-null object
nerdy            578 non-null int64
dtypes: int64(2), object(3)
memory usage: 27.1+ KB


In [341]:
selectdata.to_csv(path_or_buf='/Users/taftmphee/Desktop/selectdata.csv')

In [342]:
selectdata = pd.DataFrame.from_csv(path='/Users/taftmphee/Desktop/selectdata.csv')